In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)
import tensorflow as tf
# import tensorflow.compat.v1 as tf
tf.set_random_seed(42)

tf.disable_v2_behavior()
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

import json
from sklearn.model_selection import train_test_split

W1122 20:48:23.615469 140287136569088 deprecation.py:323] From /store/tveiga/miniconda3/envs/py/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Using TensorFlow backend.


In [2]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential, Input
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from keras.optimizers import RMSprop, Adam
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6502987162779171497
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6170955027291068975
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 18105614151092094428
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10989682688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10626947068664170743
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Gerar dataset

### - Bíblia

In [4]:
SEQUENCE_LENGTH = 80

In [5]:
with open('biblia.json') as f:
    content = f.read()

In [6]:
j = json.loads(content[1:]) # carrega string em um json

In [7]:
all_chaps = []
for book in j:
    for chap in book['chapters']:
        all_chaps.append(' '.join(chap))

In [8]:
X_data = []
y_data = []
for i in range(500_000):
    c = np.random.randint(0, len(all_chaps))
    curr_chap = all_chaps[c]
    a = np.random.randint(0, len(curr_chap) - (SEQUENCE_LENGTH + 1))
    
    X_data.append(curr_chap[a : a + SEQUENCE_LENGTH])
    y_data.append(curr_chap[a + SEQUENCE_LENGTH])

In [9]:
for dado, alvo in zip(X_data[:10], y_data[:10]):
    print(dado, '-', alvo)

ue há em Cristo Jesus com glória eterna. Fiel é esta palavra: Se, pois, já morre - m
as, vivamos no presente mundo sóbria, e justa, e piamente, aguardando a bem-aven - t
. Partindo dali, fomos navegando a sotavento de Chipre, porque os ventos eram co - n
vieram a Ezequias com as vestes rasgadas, e lhe fizeram saber as palavras de Rab - s
rolados, da idade de vinte anos e acima, que foram seiscentos e três mil quinhen - t
 moças, como a irmãs, com toda a pureza. Honra as viúvas que são verdadeiramente -  
o vinho de furor, e faze que dele bebam todas as nações, às quais eu te enviar.  - B
, e favas, e lentilhas, e milho miúdo, e espelta, e mete-os numa só vasilha, e d - e
uste será pior do que o primeiro. Disse-lhes Pilatos: Tendes uma guarda; ide, to - r
 abundância de suas delícias. Ouvi outra voz do céu dizer: Sai dela, povo meu, p - a


In [10]:
tokenizer = Tokenizer(
    num_words=500, # número máximo - valor grande arbitrário - se tirar não funciona - não sei pq
    char_level=True,
    filters=None,
    lower=False,
    oov_token=chr(1) # símbolo com quadradinho - funciona no js
)

In [11]:
%%time

tokenizer.fit_on_texts(X_data)
tokenizer.fit_on_texts(y_data)
# word_index = tokenizer.word_index
# index_word = tokenizer.index_word

print('Found %s unique tokens.' % len(tokenizer.word_index))

Found 89 unique tokens.
CPU times: user 6.27 s, sys: 0 ns, total: 6.27 s
Wall time: 6.27 s


In [12]:
# X_data_bibl40 = X_data
# y_data_bibl40 = y_data
# tok_bibl = tokenizer

## Treinar modelo

In [13]:
X = np.array(tokenizer.texts_to_sequences(X_data), dtype=np.int32)
y = np.array(list(map(tokenizer.word_index.get, y_data)))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
num_chars = len(tokenizer.index_word) + 1

In [23]:
emb_dim = 320 # 256
seq_units = 192 # 128 # faz muita diferença diminuir
drop = .1

model = Sequential()
model.add(Embedding(num_chars, emb_dim, input_length=X_train.shape[1]))
# model.add(SpatialDropout1D(drop)) # maybe should remove because of rare words...
model.add(Bidirectional(GRU(seq_units, return_sequences=True, dropout=0.1,recurrent_dropout=0.1)))
model.add(GlobalMaxPool1D())

model.add(Dense(num_chars, activation='softmax'))

In [24]:
model.compile(
    optimizer=RMSprop(lr=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 320)           28800     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 80, 384)           590976    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 384)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 90)                34650     
Total params: 654,426
Trainable params: 654,426
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
%%time
n = len(X_train) // 1
# change val split to .1 or .05?

epochs = 400
batch_size = 1024
history = model.fit(X_train[:n], y_train[:n], epochs=epochs, batch_size=batch_size,
                    validation_split=0.2,
#                     sample_weight = sample_weight[:n],
                    callbacks=[EarlyStopping(monitor='val_loss',patience=1, min_delta=1e-7)],
                   )

Train on 320000 samples, validate on 80000 samples
Epoch 1/400
320000/320000 [==============================] - 54s 169us/step - loss: 2.4540 - sparse_categorical_accuracy: 0.3093 - val_loss: 2.1755 - val_sparse_categorical_accuracy: 0.3649
Epoch 2/400
320000/320000 [==============================] - 53s 166us/step - loss: 2.0804 - sparse_categorical_accuracy: 0.3863 - val_loss: 2.0003 - val_sparse_categorical_accuracy: 0.4039
Epoch 3/400
320000/320000 [==============================] - 53s 166us/step - loss: 1.9428 - sparse_categorical_accuracy: 0.4185 - val_loss: 1.9029 - val_sparse_categorical_accuracy: 0.4276
Epoch 4/400
320000/320000 [==============================] - 53s 166us/step - loss: 1.8543 - sparse_categorical_accuracy: 0.4406 - val_loss: 1.8133 - val_sparse_categorical_accuracy: 0.4539
Epoch 5/400
320000/320000 [==============================] - 53s 167us/step - loss: 1.7865 - sparse_categorical_accuracy: 0.4598 - val_loss: 1.7576 - val_sparse_categorical_accuracy: 0.4682

### Salvar modelo

In [26]:
model.save('biblia_80-keras_model.h5')
# pickle.dump(history, open("biblia-history.p", "wb"))
with open('biblia_80-index_word.json', 'w') as f:
    json.dump(tokenizer.index_word, f, ensure_ascii=False)
with open('biblia_80-word_index.json', 'w') as f:
    json.dump(tokenizer.word_index, f, ensure_ascii=False)

In [27]:
# with open('biblia-word_index.json', 'r') as f:
#     x = json.load(f)
# x

In [28]:
# model = load_model('keras_model.h5')
# history = pickle.load(open("history.p", "rb"))

### Testar Modelo

In [29]:
curr = X_test[0].copy().reshape((1, -1))
original = ''.join(map(tokenizer.index_word.get, curr[0]))

In [30]:
next_seq = []
while True:
    _next = model.predict_classes(curr)[0]
    next_char = tokenizer.index_word[_next]
    if next_char == ' ':
        break
    next_seq.append(next_char)
    curr[0, 0 : -1] = curr[0, 1 :]
    curr[0, -1] = _next


In [31]:
print(original, ''.join(next_seq))

 se apressou a sair, porque o Senhor o ferira. Assim ficou leproso o rei Uzias a 


In [32]:
for i in range(15):
    curr = X_test[i].copy().reshape((1, -1))
    original = ''.join(map(tokenizer.index_word.get, curr[0]))
    blank_count = 0
    next_seq = []
    while True:
        _next = model.predict_classes(curr)[0]
        next_char = tokenizer.index_word[_next]
        if next_char == ' ':
            blank_count += 1
        if blank_count == 6:
            break
        next_seq.append(next_char)
        curr[0, 0 : -1] = curr[0, 1 :]
        curr[0, -1] = _next
        
    print(original, '--', ''.join(next_seq))

 se apressou a sair, porque o Senhor o ferira. Assim ficou leproso o rei Uzias a --  sua mão de Deus. E
 mulher tiver o cabelo comprido, é para ela uma glória? Pois a cabeleira lhe foi --  de todos os que estavam
 suceda que morra na peleja e outro a receba. Assim continuarão os oficiais a fa -- zer do Senhor do Senhor dos
mês, no décimo quinto ano do reinado de Asa. E no mesmo dia ofereceram em sacrif -- ícios de Israel, e os filhos
uja força era como a dos carvalhos; mas destruí o seu fruto por cima, e as suas  -- servos de todos os seus servos
iverem todos eles prontos nos teus lábios. Para que a tua confiança esteja no se -- u contra o Senhor, e o
do todos estes jóias de ouro; assim veio todo aquele que queria fazer oferta de  -- meu povo, e o seu por
o necessitado, daquele que o rouba. Levantam-se testemunhas maliciosas; interrog -- ar a minha mão de Deus.
az o coração dos tolos. O sacrifício dos ímpios é abominável ao Senhor; mas a or -- denação de todos os filhos de
 do que todo